# CRUD Concepts

## Data model


<img src="https://dl.dropboxusercontent.com/u/67572426/datamodel_sm.png">


## Connect to Neo4j

We use the [Neo4j Bolt Driver](http://neo4j.com/docs/api/python-driver/current/) to connect to Neo4j and execute Cypher queries.

In [2]:
# First we install the neo4j-driver package
!pip install neo4j-driver

In [3]:
from neo4j.v1 import GraphDatabase

In [4]:
# instantiate driver object
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "letmein"))

In [5]:
# Hello world
with driver.session() as session:
    result = session.run("MATCH (a) RETURN COUNT(a) AS num")
    
    for record in result:
        print("Number of nodes in the database: %i" % record['num'])

Number of nodes in the database: 0


## CREATE

### Create Characters



In [6]:
with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

In [7]:
# Query to create Character node
create_character_query = '''
MERGE (c:Character {name: $name})
'''

In [8]:
with driver.session() as session:
    with open("data/characters.txt") as f:
        for line in f:
            # strip whitespace
            text = line.strip()
            # execute create_character_query, passing name parameter   
            session.run(create_character_query, parameters={'name': text})

### Create Line nodes

> For each line in the text, CREATE a corresponding record in the database. Each record will include the name of the character speaking, the (absolute) line number of the phrase and the phrase itself, trimmed of any leading or following spaces

In [9]:
# Query to add a line to the database
create_line_query = '''
CREATE (l:Line {text: $line_text, number: $line_num})
WITH l
MATCH (c:Character) WHERE c.name = $current_character
CREATE (l)<-[:SPEAKS]-(c)
'''



In [10]:
# create an empty set
characters = set()

# fetch all Character names from the database
get_characters_query = '''
MATCH (c:Character)
RETURN c.name AS name
'''

# for each Character name, add it to the characters set
with driver.session() as session:
    result = session.run(get_characters_query)
    for record in result:
        characters.add(record['name'])

characters

{'BOTTOM',
 'COBWEB',
 'DEMETRIUS',
 'EGEUS',
 'FLUTE',
 'HELENA',
 'HERMIA',
 'HIPPOLYTA',
 'LYSANDER',
 'MOTH',
 'MUSTARDSEED',
 'OBERON',
 'PEASEBLOSSOM',
 'PHILOSTRATE',
 'PUCK',
 'QUINCE',
 'SNOUT',
 'SNUG',
 'STARVELING',
 'THESEUS',
 'TITANIA'}

In [11]:
# import the time 
import time

In [12]:
line_num = 0

begin = time.time()
with driver.session() as session:
    with open("data/A_Midsummer_Nights_Dream.txt") as f:
        current_character = ''
        for line in f:
            # strip leading and trailing whitespace
            text = line.strip()
            # is line the character speaking?
            if text in characters:
                current_character = text
            else:
                line_num += 1
                session.run(create_line_query, parameters={
                    'line_text': text,
                    'line_num': line_num,
                    'current_character': current_character
                })
            # insert line into database
            
    end = time.time()
    create_time = (end-begin)/line_num * 1000

print(str(create_time))
    

0.30182105813272264


## UPDATE

> For each record in the database, search for character names, convert them to UPPERCASE, then UPDATE the record in the database

In [13]:
# query to update 
update_query = '''
WITH $character AS character
WITH substring(character,0,1) + toLower(substring(character,1)) AS casedChar, character

MATCH (l:Line) WHERE toLower(l.text) CONTAINS toLower(character)
SET l.text = replace(l.text, casedChar, character)
RETURN COUNT(l) AS num
'''

In [14]:
update_count = 0

with driver.session() as session:
    begin = time.time()
    for c in characters:
        # Update the lines mentioning this character
        result = session.run(update_query, parameters = {'character': c})
        # increment update_count
        record = result.single()
        update_count += record['num']
    end = time.time()
    update_time = (end-begin)/update_count * 1000
    print(update_time)

0.44376305911851965


## DELETE

> For each record in the database, the instructor will also need to DELETE any lines that start with “ENTER” , “EXIT” , “ACT” , or “SCENE”


In [15]:
# query to delete stop words
delete_query = '''
WITH ['ENTER', 'EXIT', 'ACT', 'SCENE'] AS words
MATCH (l:Line) WHERE any(x IN words WHERE toLower(l.text) STARTS WITH toLower(x))
DETACH DELETE l
RETURN COUNT(l) AS num
'''

In [16]:
with driver.session() as session:
    begin = time.time()
    result = session.run(delete_query)
    record = result.single()
    num_lines = record['num']
    end = time.time()
    delete_time = (end-begin) / num_lines * 1000
    print(delete_time)

0.4598163976901915


## READ

When all other tasks are complete, the next step will be to READ each line and print it out to console


In [17]:
# query to read lines
read_lines_query = '''
MATCH (l:Line) RETURN l.text AS text, l.number AS linenum
'''

In [18]:
with driver.session() as session:
    begin = time.time()
    # read each line from the datbase
    result = session.run(read_lines_query)
    for record in result:
    # print each line
        print("%i: %s" % (record['linenum'], record['text']))
    end = time.time()
    
    # get number of lines in database
    result = session.run("MATCH (l:Line) RETURN COUNT(l) AS num")
    numlines = result.single()['num']
    
    
    
    read_time = (end-begin)/numlines * 1000
    print(str(read_time))

1: A Midsummer Night's Dream
2: Shakespeare homepage | Midsummer Night's Dream | Entire play
5: 
7: Now, fair HIPPOLYTA, our nuptial hour
8: Draws on apace; four happy days bring in
9: Another moon: but, O, methinks, how slow
10: This old moon wanes! she lingers my desires,
11: Like to a step-dame or a dowager
12: Long withering out a young man revenue.
13: Four days will quickly steep themselves in night;
14: Four nights will quickly dream away the time;
15: And then the moon, like to a silver bow
16: New-bent in heaven, shall behold the night
17: Of our solemnities.
18: Go, PHILOSTRATE,
19: Stir up the Athenian youth to merriments;
20: Awake the pert and nimble spirit of mirth;
21: Turn melancholy forth to funerals;
22: The pale companion is not for our pomp.
24: HIPPOLYTA, I woo'd thee with my sword,
25: And won thy love, doing thee injuries;
26: But I will wed thee in another key,
27: With pomp, with triumph and with revelling.
29: Happy be THESEUS, our renowned duke!
30: Thanks, g

885: you know, Pyramus and Thisby meet by moonlight.
886: Doth the moon shine that night we play our play?
887: A calendar, a calendar! look in the almanac; find
888: out moonshine, find out moonshine.
889: Yes, it doth shine that night.
890: Why, then may you leave a casement of the great
891: chamber window, where we play, open, and the moon
892: may shine in at the casement.
893: Ay; or else one must come in with a bush of thorns
894: and a lanthorn, and say he comes to disfigure, or to
895: present, the person of Moonshine. Then, there is
896: another thing: we must have a wall in the great
897: chamber; for Pyramus and Thisby says the story, did
898: talk through the chink of a wall.
899: You can never bring in a wall. What say you, BOTTOM?
900: Some man or other must present Wall: and let him
901: have some plaster, or some loam, or some rough-cast
902: about him, to signify wall; and let him hold his
903: fingers thus, and through that cranny shall Pyramus
904: and Thisby whispe

In [19]:
print("CREATE: %.2fms per line" % create_time)
print("READ: %.2fms per line" % read_time)
print("UPDATE %.2fms per line" % update_time)
print("DELETE %.2fms per line" % delete_time)

CREATE: 0.30ms per line
READ: 0.16ms per line
UPDATE 0.44ms per line
DELETE 0.46ms per line
